In [ ]:
import pandas as pd
from llama_cpp import Llama, llama_print_system_info
import json
from tqdm import tqdm
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score

In [2]:
corpus = pd.read_json("corpus.jsonl", lines=True)
queries = pd.read_json("queries.jsonl", lines=True)
corpus.set_index("_id", inplace=True)
queries.set_index("_id", inplace=True)
qrels = pd.read_csv("dev.tsv", sep="\t")
qrels

,query-id,corpus-id,score
0,0,620,2
1,0,621,2
2,0,622,2
3,0,616,1
4,0,617,1
...,...,...,...
4993,557,188393,1
4994,557,188394,1
4995,558,188507,2
4996,558,188508,2


In [ ]:
class LlamaJudge:
  def __init__(self, model="models/google_gemma-3-4b-it-Q4_K_M.gguf"):
    self.model = model
    self.llm = Llama(model,
      n_gpu_layers=-1,
      n_batch=32768, #16384
      n_ctx=32768, #32768
      flash_attn=True,
      rope_scaling={"type": "xpos"},
      verbose=False,
    )
  def generate_judgement(self, query, passage):
    system = """Ты — строгий эксперт по оценке качества поиска. Твоя задача — проанализировать Запрос и Отрывок текста, а затем оценить, насколько полезен этот Отрывок для ответа на Запрос.

Используй строго следующую шкалу оценки (целое число от 0 до 2):

[0] Нерелевантный: Отрывок не имеет отношения к запросу. Информация не помогает ответить на вопрос и не связана с темой даже косвенно.
[1] Частично релевантный: Отрывок тематически связан с запросом, но не содержит прямого или полного ответа. В нем может обсуждаться контекст, но ключевой информации не хватает.
[2] Полностью релевантный: Отрывок содержит прямой, точный и полный ответ на запрос. Пользователю не нужно искать дальше.

Инструкции по формату:
1. Твой ответ должен быть СТРОГО в формате JSON.
2. Используй ключ "score" и целое число (0, 1 или 2).
3. ЗАПРЕЩЕНО писать пояснения, рассуждения или любой другой текст вне JSON объекта.

Пример 1:
Запрос: "Столица Франции"
Отрывок: "Париж — столица и крупнейший город Франции, расположенный на севере страны."
Ответ: {"score": 2}

Пример 2:
Запрос: "Как приготовить борщ"
Отрывок: "Борщ — это разновидность супа, популярная в Восточной Европе. Существует много споров о его происхождении."
Ответ: {"score": 1}

Пример 3:
Запрос: "Кто написал 'Войну и мир'?"
Отрывок: "Рецепт яблочного пирога требует 1 кг яблок и 200 грамм сахара."
Ответ: {"score": 0}"""
    prompt = f"""Оцени релевантность следующей пары.

Запрос: {query}
Отрывок: {passage}

Выведи только JSON с оценкой:
"""
    response = self.llm.create_chat_completion(
      messages=[
          {"role": "system", "content": system},
          {"role": "user", "content": prompt}
      ], max_tokens=10, top_k=0, top_p=1, temperature=0,
      response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {"score": {"type": "integer"}},
            "required": ["score"],
        },
    },
    )
    return response["choices"][0]["message"]["content"]
#     response: ChatResponse = await self.client.chat(model=self.model, messages=[
#       # {
#       #   'role': 'system',
#       #   'content': system_prompt
#       # },
#       {
#         'role': 'user',
#         'content': prompt,
#       },
# ], format={
#   "type": "object",
#     "properties": {
#       "score": {
#         "type": "integer"
#       },
#     },
#     "required": [
#       "score",
#     ]}, options={
#       "num_predict": 32
#     }, think=False)
#     return response.message.content

In [4]:
judge = LlamaJudge("models/google_gemma-3-4b-it-Q4_K_M.gguf")

llama_context: n_ctx_per_seq (32768) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h6

In [5]:
results = []
test = qrels.head(50)
for idx, row in tqdm(qrels.iterrows(), total=len(qrels)):
  true_score = row["score"]
  query = queries.loc[row["query-id"]]["text"]
  response = corpus.loc[row["corpus-id"]]["text"]
  res = judge.generate_judgement(query, response)
  res = json.loads(res)
  result = {
    "llm-score": res["score"],
    "human-score": true_score,
    "query-id": row["query-id"],
    "corpus-id": row["corpus-id"]

  }
  results.append(result)

100%|██████████| 4998/4998 [1:38:28<00:00,  1.18s/it]  


In [6]:
results_df = pd.DataFrame(results)
from sklearn.metrics import cohen_kappa_score
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
cohen_kappa_score(llm, human)

0.03355374393196309

In [7]:
import numpy as np
import metrics


print("NDCG: ", metrics.calc_ndcg(results_df))
print("NRMSE: ", metrics.calc_nrmse(results_df))
print("NMAE: ", metrics.calc_nmae(results_df))
print("kendalltau: ", metrics.calc_kendalltau(results_df))
print("RBO: ", metrics.calc_rbo(results_df))

NDCG:  0.8597346124614438
NRMSE:  0.41438953591396344
NMAE:  0.34103641456582634
kendalltau:  0.5098321958450986
RBO:  0.7566482599317313


In [8]:
pd.DataFrame(results_df).to_csv('gemma3:4b_gemini.tsv', sep='\t', index=False)

In [9]:
from sklearn.metrics import confusion_matrix
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
confusion_matrix(human, llm, labels=range(3))

array([[   0,    0,    0],
       [ 783, 1330, 2546],
       [  12,   56,  271]])

In [10]:
# res = results_df[(results_df['llm-score'] == 2) & (results_df['human-score'] == 1)]
# for _, row in res.iterrows():
#   if row["query-id"] == 0:
#     continue
#   print("Query: ", queries.loc[row["query-id"]]["text"])
#   print("Passage: ", corpus.loc[row["corpus-id"]]["text"])